In [1]:
import subprocess
import re
import os

In [2]:
def get_image_name(container_image):
    match = re.search(r'/(?P<value>[^/:]+):', container_image)

    if match:
        return match.group('value')
    else:
        return "defaultName"

In [3]:
def generate_SBOM_using_syft(container_image):
    #function to generate the SBOM from a container image in CycloneDX JSON format
    #./syft ghcr.io/navisk13/actions-tester:main -o cyclonedx-json > sbom.json
    
    file_name = get_image_name(container_image)
        
    with open("sbom_" + file_name + ".json", "w") as output_file:
        subprocess.run(["../Tools/Syft/syft", container_image, "-o", "cyclonedx-json"], stdout=output_file)

In [4]:
def download_provenance_cosign(container_image):
    #function to download the provenance statement from the image on GHCR
    #./cosign download attestation ghcr.io/navisk13/actions-tester:main | jq -r '.payload' | base64 -d | jq > provenance.json
        
    file_name = get_image_name(container_image)
    
    process1 = subprocess.Popen(["../Tools/CoSign/cosign", "download", "attestation", container_image], stdout=subprocess.PIPE)
    
    process2 = subprocess.Popen(["jq", "-r", ".payload"], stdin=process1.stdout, stdout=subprocess.PIPE)
    
    process3 = subprocess.Popen(["base64", "-d"], stdin=process2.stdout, stdout=subprocess.PIPE)
    
    with open("provenance_" + file_name + ".json", "w") as output_file:
        output_file.write(process3.communicate()[0].decode())

In [5]:
def sign_metadata_ssh(file_name):
    #function to sign the artifacts using an ssh key before publishing it using rekor
    #ssh-keygen -Y sign -n file -f ../Tools/Rekor/myfirstkey -P joker sbom_actions-tester.json
    process = subprocess.Popen(["ssh-keygen", "-Y", "sign", "-n", "file", "-f", "../Tools/Rekor/myfirstkey", "-P", "joker"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)

    with open(file_name, "rb") as input_file:
        input_data = input_file.read()

    output = process.communicate(input=input_data)[0]

    with open(file_name + ".sig", "wb") as output_file:
        output_file.write(output)

In [6]:
def publish_metadata_rekor(file_name):
    #function to publish metadata on the rekor server
    #rekor-cli upload --artifact sbom_actions-tester.json --signature sbom_actions-tester.json.sig --pki-format=ssh --public-key=../Tools/Rekor/myfirstkey.pub

    signature_file = file_name + ".sig"

    process = subprocess.Popen(["rekor-cli", "upload", "--artifact", file_name, "--signature", signature_file, "--pki-format=ssh", "--public-key=../Tools/Rekor/myfirstkey.pub"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)

    with open(signature_file, "rb") as input_file:
        signature_data = input_file.read()

    output = process.communicate(input=signature_data)[0]

    artifact_url = output.decode()
    
    return artifact_url

In [7]:
def download_from_rekor_with_uuid(uuid_val, file_name):
    #download signature info from the rekor server as a JSON file
    #rekor-cli get --uuid 24296fb24b8ad77a95ce78edb06d4622b55d755ff34f0fda08ca2da724f840d863043a70b8909ea4 > signature_info.json

    process = subprocess.Popen(["rekor-cli", "get", "--uuid", uuid_val], stdout=subprocess.PIPE)

    with open(file_name + "_signature_info.json", "wb") as output_file:
        output_file.write(process.stdout.read())

In [8]:
def verify_provenance_with_slsa_verifier(container_image, repo_uri):
    IMAGE = container_image
    process = subprocess.run(["crane", "digest", IMAGE], capture_output=True, text=True)
    IMAGE = f"{IMAGE}@{process.stdout.strip()}"
    
    process = subprocess.run(["../Tools/SLSAVerifier/slsa-verifier-linux-amd64", "verify-image", IMAGE, "--source-uri", repo_uri], text=True)

In [9]:
def main():
    container_image = "ghcr.io/hongphuc-pham/metadata:main"
    repo_uri = "github.com/hongphuc-pham/metadata"
    file_name = get_image_name(container_image)
    
    if file_name != "defaultName":
    
        generate_SBOM_using_syft(container_image)
        download_provenance_cosign(container_image)

        sbom_file = "sbom_" + file_name + ".json"
        provenance_file = "provenance_" + file_name + ".json"
    
        sign_metadata_ssh(sbom_file)
        sign_metadata_ssh(provenance_file)
    
        sbom_url = publish_metadata_rekor(sbom_file)
        provenance_url = publish_metadata_rekor(provenance_file)
    
        sbom_uuid = sbom_url.split("entries/")[-1]
        sbom_uuid = sbom_uuid.strip()
    
        provenance_uuid = provenance_url.split("entries/")[-1]
        provenance_uuid = provenance_uuid.strip()
    
        download_from_rekor_with_uuid(sbom_uuid, sbom_file)
        download_from_rekor_with_uuid(provenance_uuid, provenance_file)
        
        verify_provenance_with_slsa_verifier(container_image, repo_uri)
        
    else:
        print("There was an issue retrieving the container image name, Aborting!")

In [10]:
main()

Signing data on standard input
Signing data on standard input
Verified build using builder "https://github.com/slsa-framework/slsa-github-generator/.github/workflows/generator_container_slsa3.yml@refs/tags/v1.9.0" at commit 017ff5988dbb908c6f09e82c12d9564f9f652f97
PASSED: Verified SLSA provenance
